# Utils

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

# loading and converting npy data

In [2]:
data_mel = np.load('data/npy/data_mel.npy')
labels_mel = np.load('data/npy/labels_mel.npy')

In [14]:

# Daten in Tensoren konvertieren
data_tensor = torch.from_numpy(np.array(data_mel)).float()

# import torchvision.transforms as transforms

# # Define the mean and standard deviation for normalization
# mean_mel = data_tensor.mean()
# std_mel = data_tensor.std()

# np.save('data/npy/mean_mel.npy', mean_mel)
# np.save('data/npy/std_mel.npy', std_mel)

# # Create the transform
# transform = transforms.Normalize(mean=mean_mel, std=std_mel)

# # Reshape the data_tensor to have the shape (batch_size, channels, height, width)
# data_tensor_for_normalisazion = data_tensor.view(-1, 1, 1, data_tensor.shape[1])
# # Apply the transform to the data_tensor
# normalized_data = transform(data_tensor_for_normalisazion)

labels_tensor = torch.from_numpy(np.array(labels_mel))

# Teilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)


In [15]:
# save the data
torch.save(X_train, 'data/pt/X_train.pt')
torch.save(X_test, 'data/pt/X_test.pt')
torch.save(y_train, 'data/pt/y_train.pt')
torch.save(y_test, 'data/pt/y_test.pt')

# Loading pt data

In [3]:
X_train = torch.load('data/pt/X_train.pt')
X_test = torch.load('data/pt/X_test.pt')
y_train = torch.load('data/pt/y_train.pt')
y_test = torch.load('data/pt/y_test.pt')

# Defining Model

## with class

In [9]:
# defining knn model
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        # Erstes lineares Layer
        self.fc1 = nn.Linear(22144, 5000)
        # ReLU Aktivierungsfunktion
        self.relu = nn.ReLU()
        # Dropout-Layer mit Wahrscheinlichkeit 0.2
        self.dropout = nn.Dropout(0.2)
        # Zweites lineares Layer
        self.fc2 = nn.Linear(5000, 1000)
        # ReLU Aktivierungsfunktion
        self.relu2 = nn.ReLU()
        # Drittes lineares Layer
        self.fc3 = nn.Linear(1000, 250)
        self.relu3 = nn.ReLU()
        # Dropout-Layer mit Wahrscheinlichkeit 0.2
        self.dropout2 = nn.Dropout(0.2)
        # Viertes lineares Layer
        self.fc4 = nn.Linear(250, 10)
        self.relu4 = nn.ReLU()
        # Fünftes lineares Layer
        self.fc5 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout2(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.softmax(x)
        return x



In [ ]:
# Modellinstanz
model = Model()

## with sequential

In [38]:
model = nn.Sequential(
    nn.Linear(22144, 5000),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(5000, 1000),
    nn.ReLU(),
    nn.Linear(1000, 250),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(250, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
).to(device)

### V2 of sequential

In [4]:
model = nn.Sequential(
    nn.Linear(22144, 15000),
    nn.ReLU(),
    nn.Linear(15000, 10000),
    nn.ReLU(),
    nn.Linear(10000, 5000),
    nn.ReLU(),
    nn.Linear(5000, 2500),
    nn.ReLU(),
    nn.Linear(2500, 2000),
    nn.ReLU(),
    nn.Linear(2000, 1000),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500, 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 1),
).to(device)

## loading model

In [7]:
model = torch.load('data/models/KNNmodelV1.pt')

# Defining optimizer

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.MSELoss()

# add learning rate scheduler
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

# Training model

In [10]:
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    output = model(X_train)
    output = output.squeeze()  # Remove dimensions of size 1
    loss = criterion(output, y_train.float())
    loss.backward()
    optimizer.step()
    # scheduler.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Prediction: {output[0]}, Label: {y_train[0]}')
    if loss.item() < 1-0.91:
        print('break')
        break

Epoch 1, Loss: 0.1682853400707245, Prediction: 1.2929940223693848, Label: 1
Epoch 2, Loss: 0.1674351692199707, Prediction: 1.2735421657562256, Label: 1
Epoch 3, Loss: 0.16675874590873718, Prediction: 1.261552333831787, Label: 1
Epoch 4, Loss: 0.1662050187587738, Prediction: 1.2607415914535522, Label: 1
Epoch 5, Loss: 0.16576947271823883, Prediction: 1.26121187210083, Label: 1
Epoch 6, Loss: 0.16540132462978363, Prediction: 1.253309726715088, Label: 1
Epoch 7, Loss: 0.16503269970417023, Prediction: 1.2352243661880493, Label: 1
Epoch 8, Loss: 0.16472068428993225, Prediction: 1.212985634803772, Label: 1
Epoch 9, Loss: 0.16447614133358002, Prediction: 1.196418285369873, Label: 1
Epoch 10, Loss: 0.1642206907272339, Prediction: 1.189307689666748, Label: 1
Epoch 11, Loss: 0.1640198826789856, Prediction: 1.189310908317566, Label: 1
Epoch 12, Loss: 0.1638403981924057, Prediction: 1.187508463859558, Label: 1
Epoch 13, Loss: 0.16371409595012665, Prediction: 1.1775155067443848, Label: 1
Epoch 14, 

## V2 of training model

In [40]:
from torch.utils.data import DataLoader

# Create a DataLoader to handle batching of the training data
batch_size = 32
train_data = torch.utils.data.TensorDataset(X_train, y_train.float())
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

for epoch in range(100):
    for batch_X, batch_y in train_loader:
        model.train()
        optimizer.zero_grad()
        output = model(batch_X)
        output = output.squeeze()  # Remove dimensions of size 1
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()} \t Prediction: {output[0]}, {batch_y[0]}')
    if loss.item() < 1-0.91:
        print('break')
        break

Epoch 1, Loss: 0.3177614212036133 	 Prediction: 0.14113286137580872, 1.0
Epoch 2, Loss: 0.3772405982017517 	 Prediction: 0.12729227542877197, 0.0
Epoch 3, Loss: 0.235407754778862 	 Prediction: 0.12334516644477844, 0.0
Epoch 4, Loss: 0.26235049962997437 	 Prediction: 0.14205844700336456, 0.0
Epoch 5, Loss: 0.40755367279052734 	 Prediction: 0.14328402280807495, 1.0
Epoch 6, Loss: 0.31707221269607544 	 Prediction: 0.1453307718038559, 0.0
Epoch 7, Loss: 0.34748023748397827 	 Prediction: 0.1399935930967331, 0.0
Epoch 8, Loss: 0.35070592164993286 	 Prediction: 0.1402902752161026, 1.0
Epoch 9, Loss: 0.25719547271728516 	 Prediction: 0.1458253413438797, 0.0
Epoch 10, Loss: 0.37623390555381775 	 Prediction: 0.14606483280658722, 1.0
Epoch 11, Loss: 0.40726277232170105 	 Prediction: 0.14544226229190826, 1.0
Epoch 12, Loss: 0.3475767970085144 	 Prediction: 0.14418278634548187, 0.0
Epoch 13, Loss: 0.2590540051460266 	 Prediction: 0.14226843416690826, 1.0
Epoch 14, Loss: 0.3180863857269287 	 Predict

# Evaluate model

In [11]:
# Set the model to evaluation mode
model.eval()

# Make predictions on the test data
for i in range(len(X_test)):
    with torch.no_grad():
        output = model(X_test[i])
        output = output.squeeze()
        est_loss = criterion(output, y_test[i].float())
        print(f"pred: {output.item()}, True Label: {y_test[i]}")

pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 1.0065126419067383, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 1.0110076665878296, True Label: 1
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 1.0109965801239014, True Label: 1
pred: 0.27173441648483276, True Label: 0
pred: 0.27173441648483276, True Label: 0
pred: 0.99165016412

# Safe model

In [12]:
torch.save(model, 'data/models/KNNmodelV2-2.pt')